In [1]:
import random
import numpy as np

In [2]:
# Task 1: Changed the maze by adding/removing walls
maze = [
    [0, 0, 0],
    [1, 1, 0],
    [0, 0, 0]
]

start, end = (0, 0), (2, 2)


In [9]:
n_ants = 5
n_iterations = 20
evaporation_rate = 0.1

pheromones = np.ones((len(maze), len(maze[0])))

moves = [(0,1),(0,-1),(1,0),(-1,0)]


In [10]:
def valid(x, y):
    return 0 <= x < len(maze) and 0 <= y < len(maze[0]) and maze[x][y] == 0

# Add move limit (ant dies if > 10 steps)
# heuristic (pheromone × (1/distance to goal))
def ant_walk(move_limit=10, use_heuristic=False):
    path = [start]
    visited = set([start])
    current = start
    moves_made = 0

    while current != end:
        moves_made += 1
        if moves_made > move_limit:
            return None

        x, y = current
        neighbors = [(x+dx, y+dy) for dx,dy in moves if valid(x+dx,y+dy)]
        if not neighbors:
            return None

        if use_heuristic:
            probs = [pheromones[nx][ny] * (1.0 / (abs(end[0]-nx) + abs(end[1]-ny) + 1))
                     for nx,ny in neighbors]
        else:
            probs = [pheromones[nx][ny] for nx,ny in neighbors]

        total = sum(probs)
        probs = [p/total for p in probs]
        current = random.choices(neighbors, probs)[0]

        if current in visited:
            return None

        visited.add(current)
        path.append(current)

    return path


In [11]:
def run_simulation(n_ants, use_heuristic=False, independent=False):
    global pheromones
    pheromones = np.ones((len(maze), len(maze[0])))
    best_path = None

    for it in range(1, n_iterations+1):
        print(f"\n Iteration {it}:")
        all_paths = []

        # Task 6: Independent ants option
        iteration_pheromones = np.zeros_like(pheromones)

        for a in range(1, n_ants+1):
            path = ant_walk(move_limit=10, use_heuristic=use_heuristic)
            if path:
                print(f"Ant {a}: SUCCESS (len {len(path)}) {path}")
                all_paths.append(path)

                if independent:
                    # pheromone added only after all ants finish
                    for (x,y) in path:
                        iteration_pheromones[x][y] += 1.0 / len(path)
                else:
                    for (x,y) in path:
                        pheromones[x][y] += 1.0 / len(path)
            else:
                print(f"Ant {a}: FAILED")

        if independent:
            pheromones += iteration_pheromones

        pheromones *= (1 - evaporation_rate)   # evaporation

        # print pheromone grid
        print("Pheromone grid:")
        print(np.round(pheromones, 2))

        if all_paths:
            best = min(all_paths, key=len)
            if best_path is None or len(best) < len(best_path):
                best_path = best
        print(f"Best so far: {len(best_path) if best_path else None} steps")

    print("\nFinal Best Path:", best_path)


In [6]:
# Compare with small and large numbers of ants
print(" Simulation with 2 ants:")
run_simulation(n_ants=2)

print("\n\n Simulation with 20 ants:")
run_simulation(n_ants=20)


 Simulation with 2 ants:

 Iteration 1:
Ant 1: FAILED
Ant 2: SUCCESS (len 5) [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2)]
Pheromone grid:
[[1.08 1.08 1.08]
 [0.9  0.9  1.08]
 [0.9  0.9  1.08]]
Best so far: 5 steps

 Iteration 2:
Ant 1: FAILED
Ant 2: SUCCESS (len 5) [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2)]
Pheromone grid:
[[1.15 1.15 1.15]
 [0.81 0.81 1.15]
 [0.81 0.81 1.15]]
Best so far: 5 steps

 Iteration 3:
Ant 1: FAILED
Ant 2: FAILED
Pheromone grid:
[[1.04 1.04 1.04]
 [0.73 0.73 1.04]
 [0.73 0.73 1.04]]
Best so far: 5 steps

 Iteration 4:
Ant 1: SUCCESS (len 5) [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2)]
Ant 2: FAILED
Pheromone grid:
[[1.11 1.11 1.11]
 [0.66 0.66 1.11]
 [0.66 0.66 1.11]]
Best so far: 5 steps

 Iteration 5:
Ant 1: FAILED
Ant 2: FAILED
Pheromone grid:
[[1.   1.   1.  ]
 [0.59 0.59 1.  ]
 [0.59 0.59 1.  ]]
Best so far: 5 steps

 Iteration 6:
Ant 1: FAILED
Ant 2: FAILED
Pheromone grid:
[[0.9  0.9  0.9 ]
 [0.53 0.53 0.9 ]
 [0.53 0.53 0.9 ]]
Best so far: 5 steps

 Iteration 7:


In [7]:
# Run with heuristic enabled
print("\n\n Simulation with heuristic (pheromone × 1/distance):")
run_simulation(n_ants=5, use_heuristic=True)




 Simulation with heuristic (pheromone × 1/distance):

 Iteration 1:
Ant 1: FAILED
Ant 2: FAILED
Ant 3: FAILED
Ant 4: FAILED
Ant 5: FAILED
Pheromone grid:
[[0.9 0.9 0.9]
 [0.9 0.9 0.9]
 [0.9 0.9 0.9]]
Best so far: None steps

 Iteration 2:
Ant 1: FAILED
Ant 2: FAILED
Ant 3: FAILED
Ant 4: FAILED
Ant 5: FAILED
Pheromone grid:
[[0.81 0.81 0.81]
 [0.81 0.81 0.81]
 [0.81 0.81 0.81]]
Best so far: None steps

 Iteration 3:
Ant 1: FAILED
Ant 2: FAILED
Ant 3: FAILED
Ant 4: FAILED
Ant 5: FAILED
Pheromone grid:
[[0.73 0.73 0.73]
 [0.73 0.73 0.73]
 [0.73 0.73 0.73]]
Best so far: None steps

 Iteration 4:
Ant 1: FAILED
Ant 2: FAILED
Ant 3: FAILED
Ant 4: FAILED
Ant 5: SUCCESS (len 5) [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2)]
Pheromone grid:
[[0.84 0.84 0.84]
 [0.66 0.66 0.84]
 [0.66 0.66 0.84]]
Best so far: 5 steps

 Iteration 5:
Ant 1: SUCCESS (len 5) [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2)]
Ant 2: SUCCESS (len 5) [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2)]
Ant 3: FAILED
Ant 4: SUCCESS (len 5) [(0, 0)

In [8]:
# Run with independent ants (delayed pheromone updates)
print("\n\n Simulation with independent ants (knowledge sharing after iteration):")
run_simulation(n_ants=5, use_heuristic=False, independent=True)




 Simulation with independent ants (knowledge sharing after iteration):

 Iteration 1:
Ant 1: FAILED
Ant 2: FAILED
Ant 3: FAILED
Ant 4: FAILED
Ant 5: FAILED
Pheromone grid:
[[0.9 0.9 0.9]
 [0.9 0.9 0.9]
 [0.9 0.9 0.9]]
Best so far: None steps

 Iteration 2:
Ant 1: FAILED
Ant 2: SUCCESS (len 5) [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2)]
Ant 3: FAILED
Ant 4: FAILED
Ant 5: FAILED
Pheromone grid:
[[0.99 0.99 0.99]
 [0.81 0.81 0.99]
 [0.81 0.81 0.99]]
Best so far: 5 steps

 Iteration 3:
Ant 1: FAILED
Ant 2: FAILED
Ant 3: FAILED
Ant 4: FAILED
Ant 5: FAILED
Pheromone grid:
[[0.89 0.89 0.89]
 [0.73 0.73 0.89]
 [0.73 0.73 0.89]]
Best so far: 5 steps

 Iteration 4:
Ant 1: FAILED
Ant 2: FAILED
Ant 3: FAILED
Ant 4: FAILED
Ant 5: FAILED
Pheromone grid:
[[0.8  0.8  0.8 ]
 [0.66 0.66 0.8 ]
 [0.66 0.66 0.8 ]]
Best so far: 5 steps

 Iteration 5:
Ant 1: FAILED
Ant 2: FAILED
Ant 3: FAILED
Ant 4: FAILED
Ant 5: FAILED
Pheromone grid:
[[0.72 0.72 0.72]
 [0.59 0.59 0.72]
 [0.59 0.59 0.72]]
Best so far: 5 step